In [ ]:
import numpy as np

In [ ]:
def parse(data):
    '''
    parse(data) computes the necessary features to classify
    data MUST be a 101 row x 6 column array
    columns are t, a, e, i, Omega, omega
    rows are different time outputs: MUST be 1000yr outputs, ie [0, 1E3, 2E3....99E3,100E3]
    Returns features for classification
    '''
    
    # Take stats of simulations
    initials = data[0,1:] # a, e, i, Omega, omega
    finals = data[-1,1:]
    
    mins = np.amin(data[:,1:],axis = 0)
    maxes = np.amax(data[:,1:],axis = 0)
    dels = maxes-mins
    
    means = np.mean(data[:,1:],axis = 0)
    stdev = np.std(data[:,1:],axis = 0)
    

    # Take time derivatives
    diffs = data[1:,:]-data[:-1,:]
    dxdt = diffs[:,1:]/diffs[:,0, np.newaxis] # add on new axis to time to give same dimensionality as the numerator
      
    mindxdt = np.amin(dxdt,axis = 0)
    meandxdt = np.mean(dxdt,axis = 0)
    maxdxdt = np.amax(dxdt,axis = 0)
    deldxdt = maxdxdt-mindxdt

    # rearrange data into the order I want
    arrs = [initials,finals,mins,means,maxes,stdev,dels,mindxdt,meandxdt,maxdxdt,deldxdt]
    inds = [0,1,2,3,4] # a, e, i, Omega, omega
    features = []
    
    ## features contains all x values, then all y, etc: xi, xf, xmin, xmean, xmax, xsigma, Deltax, xdotmin, xdotmean, xdotmax 
    for i in inds:
        for a in arrs:
            features += [a[i]]

    return np.array(features).reshape(1, -1) # make sure features is a 2d array 

In [ ]:
def start_simulation(epoch):
    '''
    Initialize rebound simulation and add in Sun+giant planets
    epoch must be in HJD, eg 2453318.5
    Returns simulation object
    '''
    
    import rebound
    
    # start simulation and set useful units
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # format epoch to correct format
    if epoch > 0: date = 'JD'+str(round(float(epoch),4)).ljust(12,'0')
    else: date = None
    # add in Sun and giant planets
    sim.add("Sun", date=date)
    sim.add("Jupiter", date=date)
    sim.add("Saturn", date=date)
    sim.add("Uranus", date=date)
    sim.add("Neptune", date=date)
    
    return sim

In [ ]:
def run_simulation(sim,savename=""):
    '''
    Run the simulation and output data at correct intervals
    Outputs simulation file savename if defined
    Returns simulation and data from which features are computed
    '''
    
    # Open file if necessary
    writeout=False
    if savename!="": writeout=True
    if writeout: 
        file=open(savename,'a')
        file.truncate(0) # delete content of file if it already exists
        file.write('id, t, a, e, i, Omega, omega, M\n')
        
    # Make expected time outputs
    time_outs = np.linspace(0,100E3,101)
    
    # Set KBO as test particle
    sim.N_active = sim.N - 1  

    # Set useful simulation settings
    sim.move_to_com()
    sim.integrator = 'mercurius' 
    sim.dt = 0.1 #timestep in years
    
    # Integrate and save outputs
    data = []
    for i,t in enumerate(time_outs):
        
        if t>0: sim.integrate(t, exact_finish_time=True) # integrate to next output
        
        # Output orbits 
        orbits = sim.calculate_orbits(primary=sim.calculate_com())
        o = orbits[-1] # take KBO
        step = np.array([t, o.a, o.e, np.degrees(o.inc), np.degrees(o.Omega)%360, np.degrees(o.omega)%360]) # save t, a, e, i, Omega, omega

        # add step to data
        if len(data)==0: data = step
        else: data = np.vstack((data,step))
            
        # write out step for Neptune and KBO test particle
        if writeout: 
            
            o_N=orbits[-2] # take Neptune
            N_line=[int(-1), t, o_N.a, o_N.e, np.degrees(o_N.inc), np.degrees(o_N.Omega)%360, np.degrees(o_N.omega)%360, np.degrees(o_N.M)%360]
            file.write(' '.join(map(str,N_line))); file.write('\n')

            o=orbits[-1] # take KBO
            KBO_line=[int(0), t, o.a, o.e, np.degrees(o.inc), np.degrees(o.Omega)%360, np.degrees(o.omega)%360, np.degrees(o.M)%360]
            file.write(' '.join(map(str,KBO_line))); file.write('\n')

    if writeout: file.close()
        
    return sim, data

In [ ]:
def compute_from_file(fname, col_order=[0,1,2,3,4,5]):
    '''
    Load data from file and calculate features
    fname=file name that contains simulation data; MUST contain outputs only every 1000 years, only contain lines from expected object
      Will only take first 101 lines from fname, which should be times [0,1E3,2E3,...,99E3,100E3]
    col_order=indexes for columns: time, a (semi-major axis), eccentriciy, inclination, Omega (longitude of ascending node), omega (argument of pericenter)
    Returns features for ML classification
    '''
    
    # Load fname
    data_t = np.loadtxt(fname)
    data_t=data_t[:101,:]

    # sort columns based on col_order
    co = col_order
    data = np.array(data_t[:,co[0]])[:,np.newaxis]
    for i in co[1:]:
        data = np.concatenate((data,data_t[:,co[i]][:,np.newaxis]), axis=1)

    print('Loaded',fname)
    print()
   
    # Compute features
    features = parse(data)
    return features

#compute_from_file()

In [ ]:
def compute_from_aei(epoch=2453318.8875, a=39.403, ecc=0.20724, inc=5.7449, Omega=44.431, omega=291.10, M=45.471,barycentric=False,savename=""):
    '''
    Run simulation and calculate features for an object specified by epoch, a, e, i, Omega, omega, and M
    epoch must be in HJD, eg 2453318.5. epoch=0 uses today's values
    orbital elements are heliocentric
        -a in AU
        -i, Omega, omega, M in degrees
    use barycentric=True if the orbital elements are in barycentric coordinates
    savename is a filename to save the simulation output

    Will add giant planets to simulation from JPL Horizons database on date from epoch
    Default values are for KBO K04VD0X
    Returns features for ML classification
    '''

    # Initialize simulation and add giant planets
    sim = start_simulation(epoch)
    
    # Shift giant planets to barycentric if flag is set
    if barycentric: primary = sim.calculate_com()
    else: primary=None
    # Add KBO
    sim.add(a=a, e=ecc, inc=np.radians(inc), omega=np.radians(omega), Omega=np.radians(Omega), M=np.radians(M), primary=None)
 
    # Run simulation and print helpful stats
    E0 = sim.calculate_energy()
    sim,data = run_simulation(sim,savename)
    sim.status()
    print('Relative Energy Error (ΔE/E0):', abs((sim.calculate_energy()-E0)/E0))
    print()
    
    # Compute features for ML classification
    features = parse(data)
    return features

#compute_from_aei()

In [ ]:
def compute_from_jpl(epoch=0., objname='K04VD0X',savename=""):
    '''
    Run simulation and calculate features for an object specified by JPL Horizons identifier
    epoch, if passed, must be in HJD, eg 2453318.5, otherwise defaults to today
    savename is a filename to save the simulation output

    Will add giant planets to simulation from JPL Horizons database on date from epoch
    Default values are for KBO K04VD0X
    Returns features for ML classification
    '''
    
    # Initialize simulation and add giant planets
    sim = start_simulation(epoch)
    
    # Add KBO
    if epoch > 0: date = 'JD'+str(round(float(epoch),4)).ljust(12,'0')
    else: date = None
    sim.add('NAME='+objname, date=date)
 
    # Run simulation and print helpful stats
    E0 = sim.calculate_energy()
    sim,data = run_simulation(sim,savename)
    sim.status()
    print('Relative Energy Error (ΔE/E0):', abs((sim.calculate_energy()-E0)/E0))
    print()
    
    # Compute features for ML classification
    features = parse(data)
    return features

#compute_from_jpl()